MIT License

Copyright (c) 2025 Jędrzej Kubica

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# How to use this notebook

This notebook is divided into two parts:
Part 1. Exploring recombination map from Palsson 2024
Part 2. Exploring recombination map from Halldorsson 2019

In [ ]:
import pandas
import numpy
from scipy.ndimage import gaussian_filter1d

import matplotlib.pyplot

In [ ]:
DATA_DIR = "/home/kubicaj/calc/Elixir-BH-2025/data/"
SAVE_FIG = False
FIGURES_DIR = "/home/kubicaj/calc/Elixir-BH-2025/figures/"

# Palsson 2024

In [ ]:
RECOMBINATION_MAP_MAT = DATA_DIR+"DecodeGenetics/DecodeGenetics-PalssonEtAl_Nature_2024-8e49794/data/maps/maps.mat.tsv"
RECOMBINATION_MAP_PAT = DATA_DIR+"DecodeGenetics/DecodeGenetics-PalssonEtAl_Nature_2024-8e49794/data/maps/maps.pat.tsv"

In [ ]:
mat = pandas.read_csv(RECOMBINATION_MAP_MAT,
                      sep="\t",
                      comment="#",
                      dtype={"Chr": str,
                             "pos": numpy.uint32,
                             "map": numpy.float32,
                             "cMperMb": numpy.float32,
                             "DSB": numpy.float32,
                             "deltaDSB": numpy.float32,
                             "oNCO": numpy.float32})  

mat.head()

In [ ]:
# use only one chromosome for now
CHR = "chr6"

mat_chr = mat[mat["Chr"] == CHR]
mat_chr.reset_index(inplace=True, drop=True)

mat_chr.head()

In [ ]:
# use DSB (double-strand breaks, DSBs/Mb per meiosis),
# PRDM9-driven DSBs correspond strongly with recombination hotspots
recomb_rates = mat_chr["DSB"].to_numpy()

In [ ]:
# plot map, cMperMb, DSB, deltaDSB, oNCO from maternal map
fig, axs = matplotlib.pyplot.subplots(5, 1, figsize=(10, 15), sharex=True)
axs[0].scatter(x=mat_chr["pos"].to_numpy(), y=mat_chr["map"].to_numpy(), color="blue")
axs[0].set_ylabel("map (cM)")
axs[0].set_title(f"deCODE recombination map - {CHR} maternal")
axs[0].ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
axs[1].scatter(x=mat_chr["pos"].to_numpy(), y=mat_chr["cMperMb"].to_numpy(), color="orange")
axs[1].set_ylabel("cMperMb")
axs[2].scatter(x=mat_chr["pos"].to_numpy(), y=mat_chr["DSB"].to_numpy(), color="green")
axs[2].set_ylabel("DSB")
axs[3].scatter(x=mat_chr["pos"].to_numpy(), y=mat_chr["deltaDSB"].to_numpy(), color="red")
axs[3].set_ylabel("deltaDSB")
axs[4].scatter(x=mat_chr["pos"].to_numpy(), y=mat_chr["oNCO"].to_numpy(), color="purple")
axs[4].set_ylabel("oNCO")
axs[4].set_xlabel(f"{CHR} position (bp)")
axs[4].ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xticks(rotation=70)
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_all_columns.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# plot on one plot map, cMperMb, DSB, deltaDSB, oNCO from maternal map, normalize all to 0-1 range
fig, ax = matplotlib.pyplot.subplots(1, 1, figsize=(10, 5), sharex=True)
ax.scatter(x=mat_chr["pos"].to_numpy(), y=(mat_chr["map"].to_numpy() - mat_chr["map"].to_numpy().min()) / (mat_chr["map"].to_numpy().max() - mat_chr["map"].to_numpy().min()), color="blue", label="map (cM)")
ax.scatter(x=mat_chr["pos"].to_numpy(), y=(mat_chr["cMperMb"].to_numpy() - mat_chr["cMperMb"].to_numpy().min()) / (mat_chr["cMperMb"].to_numpy().max() - mat_chr["cMperMb"].to_numpy().min()), color="orange", label="cMperMb")
ax.scatter(x=mat_chr["pos"].to_numpy(), y=(mat_chr["DSB"].to_numpy() - mat_chr["DSB"].to_numpy().min()) / (mat_chr["DSB"].to_numpy().max() - mat_chr["DSB"].to_numpy().min()), color="green", label="DSB")
ax.scatter(x=mat_chr["pos"].to_numpy(), y=(mat_chr["deltaDSB"].to_numpy() - mat_chr["deltaDSB"].to_numpy().min()) / (mat_chr["deltaDSB"].to_numpy().max() - mat_chr["deltaDSB"].to_numpy().min()), color="red", label="deltaDSB")
ax.scatter(x=mat_chr["pos"].to_numpy(), y=(mat_chr["oNCO"].to_numpy() - mat_chr["oNCO"].to_numpy().min()) / (mat_chr["oNCO"].to_numpy().max() - mat_chr["oNCO"].to_numpy().min()), color="purple", label="oNCO")
ax.set_ylabel("normalized to 0-1 range")
ax.set_title(f"deCODE recombination map - {CHR} maternal")
ax.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
ax.set_xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_all_columns_one_plot_norm.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# calculate correlation matrix between map, cMperMb, DSB, deltaDSB, oNCO from maternal map
corr_matrix = mat_chr[["map", "cMperMb", "DSB", "deltaDSB", "oNCO"]].corr()
print(corr_matrix)

# plot correlation matrix as heatmap
fig, ax = matplotlib.pyplot.subplots(1, 1, figsize=(8, 6))
cax = ax.matshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ax.set_xticks(numpy.arange(len(corr_matrix.columns)))
ax.set_yticks(numpy.arange(len(corr_matrix.columns)))
ax.set_xticklabels(corr_matrix.columns)
ax.set_yticklabels(corr_matrix.columns)
matplotlib.pyplot.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        ax.text(j, i, f"{corr_matrix.iloc[i, j]:.2f}", ha="center", va="center", color="black")
ax.set_title(f"Correlation matrix - {CHR} maternal")
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_all_columns_corr.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# compare maternal and paternal DSB maps
pat = pandas.read_csv(RECOMBINATION_MAP_PAT,
                      sep="\t",
                      comment="#",
                      dtype={"Chr": str,
                             "pos": numpy.uint32,
                             "map": numpy.float32,
                             "cMperMb": numpy.float32,
                             "DSB": numpy.float32,
                             "deltaDSB": numpy.float32,
                             "oNCO": numpy.float32})

pat_chr = pat[pat["Chr"] == CHR]

pat_chr.reset_index(inplace=True, drop=True)
pat_recomb_rates = pat_chr["DSB"].to_numpy()

matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(), y=recomb_rates
                          , color="blue", label="maternal")
matplotlib.pyplot.scatter(x=pat_chr["pos"].to_numpy(), y=pat_recomb_rates
                            , color="orange", label="paternal")
matplotlib.pyplot.title("deCODE recombination map")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
       matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_vs_pat.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# find positions with recombination rate > 2*avg
avg_recomb_rate = numpy.average(recomb_rates)
high_recomb_pos = mat_chr["pos"][mat_chr["DSB"] > 2 * avg_recomb_rate].to_numpy()
high_recomb_rates = mat_chr["DSB"][mat_chr["pos"].isin(high_recomb_pos)].to_numpy()

In [ ]:
# plot the recombination rates along the chromosome,
# identify haploblock boundaries as recombination rate > 2*avg
matplotlib.pyplot.figure()

matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(), y=recomb_rates, color="grey")
matplotlib.pyplot.axhline(avg_recomb_rate, color="red", label="avg rate")

# color red the high recombination positions
matplotlib.pyplot.scatter(x=high_recomb_pos,
                          y=high_recomb_rates,
                          color="red",
                          label="rate > 2*avg")

matplotlib.pyplot.title("deCODE recombination map (maternal)")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)

matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_outliersAVG.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
print(len(high_recomb_pos), "high recombination positions (rate > 2*avg):")
for pos, rate in zip(high_recomb_pos, high_recomb_rates):
    print(f"{CHR}:{pos}\trecombination rate: {rate} DSBs/Mb per meiosis")

In [ ]:
# plot the recombination rate distribution,
# identify haploblock boundaries as recombination rate > 1.5*IQR
matplotlib.pyplot.figure()
matplotlib.pyplot.boxplot(recomb_rates, vert=True)

matplotlib.pyplot.title("deCODE recombination map (maternal)")
matplotlib.pyplot.ylabel("recombination rate")
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_boxplot.png", bbox_inches='tight')
matplotlib.pyplot.show()

# get top outliers (rate > 1.5*IQR)
q75, q25 = numpy.percentile(recomb_rates, [75, 25])
iqr = q75 - q25
upper_bound = q75 + (1.5 * iqr)
lower_bound = q25 - (1.5 * iqr)
outliers = mat_chr["pos"][ (recomb_rates > upper_bound) | (recomb_rates < lower_bound) ].to_numpy()
outlier_rates = mat_chr["DSB"][ (recomb_rates > upper_bound) | (recomb_rates < lower_bound) ].to_numpy()

matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(),
                          y=recomb_rates,
                          color="grey")
matplotlib.pyplot.scatter(x=outliers,
                          y=outlier_rates,
                          color="red",
                          label="rate > 1.5*IQR")
matplotlib.pyplot.axhline(upper_bound, color="red", label="1.5*IQR")
matplotlib.pyplot.title("deCODE recombination map (maternal)")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_outliersIQR.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# print haploblock boundaries
high_recomb_pos = outliers
high_recomb_rates = mat_chr["DSB"][mat_chr["pos"].isin(high_recomb_pos)].to_numpy()
print(len(high_recomb_pos), "high recombination positions (rate > 1.5*IQR):")
for pos, rate in zip(high_recomb_pos, high_recomb_rates):
    print(f"{CHR}:{pos}\trecombination rate: {rate} DSBs/Mb per meiosis")

In [ ]:
# identify haploblock boundaries as recombination rate > 1.5*avg of surroundings +-2.5Mb
high_recomb_pos = []
high_recomb_rates = []

for i in range(len(recomb_rates)):
    # take average of surroundings +-2.5Mb
    surroundings_average = numpy.average(recomb_rates[max(0, i-25):min(len(recomb_rates), i+25)])
    if recomb_rates[i] > 1.5 * surroundings_average:
        high_recomb_pos.append(mat_chr["pos"][i])
        high_recomb_rates.append(recomb_rates[i])
        print(f"High recombination rate at {CHR}:{mat_chr['pos'][i]}: {recomb_rates[i]} DSBs/Mb per meiosis (avg of surroundings: {surroundings_average})")

In [ ]:
# plot the recombination rates along the chromosome with haploblock boundaries
matplotlib.pyplot.figure()

matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(), y=recomb_rates, color="grey")

# color red the high recombination positions
matplotlib.pyplot.scatter(x=high_recomb_pos,
                          y=high_recomb_rates,
                          color="red",
                          label="rate > 1.5 * avg_surroundings(+-2.5Mb)")

matplotlib.pyplot.title("deCODE recombination map (maternal)")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)

matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_outliersAVGsurroundings.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# identify haploblock boundaries using peaks of recombination rates after Gaussian smoothing
recomb_rates_smoothed = gaussian_filter1d(recomb_rates,
                                          sigma=5) # std dev for kernel

# find positions corresponding to peaks in the smoothed recombination rates
peaks = (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[:-2]) & (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[2:])
peak_positions = mat_chr["pos"].to_numpy()[1:-1][peaks]

In [ ]:
# plot peaks on the smoothed recombination rates
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(), y=recomb_rates_smoothed, color="blue")
matplotlib.pyplot.scatter(x=peak_positions,
                          y=recomb_rates_smoothed[1:-1][peaks],
                          color="red",
                          label="peaks")
matplotlib.pyplot.title("deCODE recombination map (maternal) - smoothed with peaks")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_GS_peaks.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# compare different sigma for Gaussian smoothing
# plot the orginial and smoothed recombination rates with different sigma
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=mat_chr["pos"].to_numpy(), y=recomb_rates, color="grey", label="original")
for sigma in [1, 3, 5, 10]:
    recomb_rates_smoothed = gaussian_filter1d(recomb_rates, sigma=sigma)
    matplotlib.pyplot.plot(mat_chr["pos"].to_numpy(), recomb_rates_smoothed, label=f"sigma={sigma}", linewidth=3)
matplotlib.pyplot.title("deCODE recombination map (maternal) - Gaussian smoothing with different sigma")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("recombination rate")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_mat_GS_compare_sigma.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# repeat Gaussian smoothing for all chromosomes, plot peaks
chromosomes = sorted(mat["Chr"].unique())  # sort by chr number, not lexicographically
fig, axs = matplotlib.pyplot.subplots(len(chromosomes), 1, figsize=(12, 3 * len(chromosomes)), sharex=False)

for i, chrom in enumerate(chromosomes):
    mat_chr = mat[mat["Chr"] == chrom]
    recomb_rates = mat_chr["DSB"].to_numpy()
    positions = mat_chr["pos"].to_numpy()
    recomb_rates_smoothed = gaussian_filter1d(recomb_rates, sigma=5)
    peaks = (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[:-2]) & (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[2:])
    peak_positions = mat_chr["pos"].to_numpy()[1:-1][peaks]
    
    axs[i].scatter(positions, recomb_rates, color="grey", alpha=0.5, label="original")
    axs[i].scatter(positions, recomb_rates_smoothed, color="blue", label="smoothed (sigma=5)")
    axs[i].scatter(x=peak_positions,
                          y=recomb_rates_smoothed[1:-1][peaks],
                          color="red",
                          label="peaks")
    axs[i].set_title(f"{chrom}")
    axs[i].set_ylabel("recomb. rate")
    axs[i].ticklabel_format(useOffset=False, style='plain')
    axs[i].legend()

axs[-1].set_xlabel("position (bp)")
matplotlib.pyplot.tight_layout()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_all_chromosomes_GS_peaks.png", bbox_inches='tight')
matplotlib.pyplot.show()

# Part 2: Halldorsson 2019

In [ ]:
RECOMBINATION_MAP = DATA_DIR+"Halldorsson2019/aau1043_datas3"

In [ ]:
map = pandas.read_csv(RECOMBINATION_MAP,
                      sep="\t",
                      comment="#",
                      dtype={"Chr": str,
                             "Begin": numpy.uint32,
                             "End": numpy.uint32,
                             "cMperMb": numpy.float32,
                             "CM": numpy.float32})  

map.head()

In [ ]:
# use only one chromosome for now
CHR = "chr6"

map_chr = map[map["Chr"] == CHR]
map_chr.reset_index(inplace=True, drop=True)

# prepare table for github readme
map_chr.head()

In [ ]:
# use cMperMb
recomb_rates = map_chr["cMperMb"].to_numpy()

In [ ]:
# plot cMperMb from Halldorsson 2019 map
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=(map_chr["Begin"].to_numpy() + map_chr["End"].to_numpy()) // 2, y=recomb_rates, color="blue")
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR}")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("cMperMb")
matplotlib.pyplot.xticks(rotation=70)
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_Halldorsson2019_cMperMb.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# find positions with recombination rate > 10*avg
avg_recomb_rate = numpy.average(recomb_rates)
high_recomb_pos = map_chr["Begin"][map_chr["cMperMb"] > 10 * avg_recomb_rate].to_numpy()
high_recomb_rates = map_chr["cMperMb"][map_chr["Begin"].isin(high_recomb_pos)].to_numpy()
print(len(high_recomb_pos), "high recombination positions (rate > 10*avg):")
for pos, rate in zip(high_recomb_pos, high_recomb_rates):
    print(f"{CHR}:{pos}\trecombination rate: {rate} cMperMb")

In [ ]:
# save haploblock boundaries to TSV file
with open(DATA_DIR+f"haploblock_boundaries_{CHR}.tsv", "w") as f:
    f.write(f"START\tEND\n")
    for i in range(len(high_recomb_pos)):
        pos = high_recomb_pos[i]
        if i+1 < len(high_recomb_pos):
            pos1 = high_recomb_pos[i+1]
        else:
            continue
        f.write(f"{pos}\t{pos1}\n")

In [ ]:
# find positions with recombination rate > 1.5*IQR
matplotlib.pyplot.figure()
matplotlib.pyplot.boxplot(recomb_rates, vert=True)
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR}")
matplotlib.pyplot.ylabel("cMperMb")
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_Halldorsson2019_boxplot.png", bbox_inches='tight')
matplotlib.pyplot.show()

# get top outliers (rate > 1.5*IQR)
q75, q25 = numpy.percentile(recomb_rates, [75, 25])
iqr = q75 - q25
upper_bound = q75 + (1.5 * iqr)
lower_bound = q25 - (1.5 * iqr)
outliers = map_chr["Begin"][ (recomb_rates > upper_bound) | (recomb_rates < lower_bound) ].to_numpy()
outlier_rates = map_chr["cMperMb"][ (recomb_rates > upper_bound) | (recomb_rates < lower_bound) ].to_numpy()

print(len(outliers), "high recombination positions (rate > 1.5*IQR):")
for pos, rate in zip(outliers, outlier_rates):
    print(f"{CHR}:{pos}\trecombination rate: {rate} cMperMb")

matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=(map_chr["Begin"].to_numpy() + map_chr["End"].to_numpy()) // 2, y=recomb_rates, color="grey")
matplotlib.pyplot.axhline(upper_bound, color="red", label="1.5*IQR")
matplotlib.pyplot.scatter(x=outliers,
                          y=outlier_rates,
                          color="red",
                          label="rate > 1.5*IQR")
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR} with outliers")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("cMperMb")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_Halldorsson2019_outliersIQR.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# identify haploblock boundaries using peaks of recombination rates after Gaussian smoothing
recomb_rates_smoothed = gaussian_filter1d(recomb_rates,
                                          sigma=5) # std dev for kernel

# find positions corresponding to peaks in the smoothed recombination rates
peaks = (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[:-2]) & (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[2:])
peak_positions = map_chr["Begin"].to_numpy()[1:-1][peaks]

print(len(peak_positions), "peaks found using Gaussian smoothing (sigma=5)")

In [ ]:
# plot peaks on the smoothed recombination rates
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=(map_chr["Begin"].to_numpy() + map_chr["End"].to_numpy()) // 2, y=recomb_rates_smoothed, color="blue")
matplotlib.pyplot.scatter(x=peak_positions,
                          y=recomb_rates_smoothed[1:-1][peaks],
                          color="red",
                          label="peaks")
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR} smoothed with peaks")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("cMperMb")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"recomb_map_{CHR}_Halldorsson2019_GS_peaks.png", bbox_inches='tight')
matplotlib.pyplot.show()

# zoom in on region of 1 MBp around the first peak, mark all peaks in the region
# plot the original and smoothed recombination rates with peaks
region_start = max(0, peak_positions[0] - 500000)
region_end = peak_positions[0] + 500000
region_mask = (map_chr["Begin"] >= region_start) & (map_chr["End"] <= region_end)
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=(map_chr["Begin"][region_mask].to_numpy() + map_chr["End"][region_mask].to_numpy()) // 2,
                          y=recomb_rates[region_mask],
                          color="grey", label="original")
recomb_rates_smoothed_region = gaussian_filter1d(recomb_rates[region_mask], sigma=5)
matplotlib.pyplot.plot((map_chr["Begin"][region_mask].to_numpy() + map_chr["End"][region_mask].to_numpy()) // 2,
                       recomb_rates_smoothed_region,
                       color="blue", label="smoothed (sigma=5)", linewidth=3)
# mark all peaks in the region
peaks_region = (recomb_rates_smoothed_region[1:-1] > recomb_rates_smoothed_region[:-2]) & (recomb_rates_smoothed_region[1:-1] > recomb_rates_smoothed_region[2:])
peak_positions_region = (map_chr["Begin"][region_mask].to_numpy() + map_chr["End"][region_mask].to_numpy()) // 2
matplotlib.pyplot.scatter(x=peak_positions_region[1:-1][peaks_region],
                          y=recomb_rates_smoothed_region[1:-1][peaks_region],
                          color="red",
                          label="peaks")
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR} zoom in on {region_start}-{region_end}")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("cMperMb")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"figures/recomb_map_{CHR}_Halldorsson2019_GS_peaks_zoom_in.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# compare different sigma for Gaussian smoothing
# plot the orginial and smoothed recombination rates with different sigma
# zoom in on region of 1 MBp around the first peak
region_start = max(0, peak_positions[0] - 500000)
region_end = peak_positions[0] + 500000
region_mask = (map_chr["Begin"] >= region_start) & (map_chr["End"] <= region_end)
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(x=(map_chr["Begin"][region_mask].to_numpy() + map_chr["End"][region_mask].to_numpy()) // 2,
                          y=recomb_rates[region_mask],
                          color="grey", label="original")
for sigma in [1, 3, 5, 10]:
    recomb_rates_smoothed = gaussian_filter1d(recomb_rates[region_mask], sigma=sigma)
    matplotlib.pyplot.plot((map_chr["Begin"][region_mask].to_numpy() + map_chr["End"][region_mask].to_numpy()) // 2,
                           recomb_rates_smoothed,
                           label=f"sigma={sigma}", linewidth=3)
matplotlib.pyplot.title(f"Halldorsson 2019 recombination map - {CHR} zoom in on {region_start}-{region_end} with different sigma")
matplotlib.pyplot.ticklabel_format(useOffset=False, style='plain')  # disable scientific notation
matplotlib.pyplot.xlabel(f"{CHR} position (bp)")
matplotlib.pyplot.ylabel("cMperMb")
matplotlib.pyplot.xticks(rotation=70)
matplotlib.pyplot.legend()
if SAVE_FIG:
    matplotlib.pyplot.savefig(FIGURES_DIR+f"figures/recomb_map_{CHR}_Halldorsson2019_GS_compare_sigma_zoom_in.png", bbox_inches='tight')
matplotlib.pyplot.show()

In [ ]:
# print the number of peaks found with different sigma,
# create a table for GitHub markdown
print("\n| Sigma | Number of peaks |")
print("|-------|-----------------|")
for sigma in [1, 3, 5, 10]:
    recomb_rates_smoothed = gaussian_filter1d(recomb_rates, sigma=sigma)
    peaks = (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[:-2]) & (recomb_rates_smoothed[1:-1] > recomb_rates_smoothed[2:])
    peak_positions = map_chr["Begin"].to_numpy()[1:-1][peaks]
    print(f"| {sigma} | {len(peak_positions)} |")